In [1]:

import numpy as np
import pandas as pd
import _pickle as pickle
#from collections import defaultdict
#import re
#import ast

# from bs4 import BeautifulSoup

import sys
import os
import json
import operator

#from keras.preprocessing.text import Tokenizer, text_to_word_sequence
#from keras.preprocessing.sequence import pad_sequences
#from keras.utils.np_utils import to_categorical

#from keras import backend as K
#from keras.engine.topology import Layer, InputSpec
#from keras import initializers

#from keras.optimizers import SGD

#from keras.layers import Embedding, Dense, Conv1D, MaxPooling1D, Dropout, Activation, Input, Flatten, Concatenate, GlobalAveragePooling1D, LSTM, GRU, Bidirectional, dot, multiply, Lambda, TimeDistributed, Masking

#from keras.models import Model, Sequential
#from keras.regularizers import l2,l1,l1_l2
#from keras.callbacks import Callback,EarlyStopping, ModelCheckpoint
from sklearn.utils import class_weight

#from keras.callbacks import Callback


#from keras.layers.normalization import BatchNormalization
from sklearn.svm import SVC, LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, precision_recall_curve, classification_report,accuracy_score, auc, roc_curve, roc_auc_score

import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# from collections import OrderedDict
# from lime.lime_text import LimeTextExplainer 
# from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction import text

In [3]:
from sklearn.metrics import average_precision_score

In [4]:
import pickle

In [5]:
import optunity.metrics

In [6]:
from scipy import sparse

In [7]:
def load_data(data_name):
    f = open(data_name,'rb')
    data = pickle.load(f)
    return data

In [9]:
data = load_data('data_20200212_2012')

In [11]:
type(data)

list

In [14]:
data[0]

(['  the following discussion should be read in conjunction with , and is qualified in its entirety by , the consolidated financial statements and notes thereto included elsewhere in this annual report on form dd . this item contains forward - looking statements that involve risks and uncertainties . actual results may differ materially from those indicated in such forward - looking statements . factors that may cause such a difference include , but are not limited to , those discussed in " item dd : business -- factors affecting the company \'s business , operating results and financial condition . "',
  'j.d . edwards develops , markets and supports highly functional enterprise resource planning ( " erp " ) software for managing the supply chain . the company provides the core software products to run an entire business . these software solutions operate on multiple computing platforms and are designed to deliver the solutions that organizations need to maintain control of their busi

In [21]:
text_para = []
labels = []
for d in data:
    text_para.append(d[0])
    labels.append(d[1])

In [38]:
with open('label_20200214_2012', 'wb') as fp:
     pickle.dump(labels, fp)
fp.close()

In [22]:
text_para[0]

['  the following discussion should be read in conjunction with , and is qualified in its entirety by , the consolidated financial statements and notes thereto included elsewhere in this annual report on form dd . this item contains forward - looking statements that involve risks and uncertainties . actual results may differ materially from those indicated in such forward - looking statements . factors that may cause such a difference include , but are not limited to , those discussed in " item dd : business -- factors affecting the company \'s business , operating results and financial condition . "',
 'j.d . edwards develops , markets and supports highly functional enterprise resource planning ( " erp " ) software for managing the supply chain . the company provides the core software products to run an entire business . these software solutions operate on multiple computing platforms and are designed to deliver the solutions that organizations need to maintain control of their busine

In [23]:
def generate_doc(x):
    return ' '.join(x)

In [24]:
text_doc = []
for d in text_para:
    text_doc.append(generate_doc(d))

In [25]:
text_doc[0]

'  the following discussion should be read in conjunction with , and is qualified in its entirety by , the consolidated financial statements and notes thereto included elsewhere in this annual report on form dd . this item contains forward - looking statements that involve risks and uncertainties . actual results may differ materially from those indicated in such forward - looking statements . factors that may cause such a difference include , but are not limited to , those discussed in " item dd : business -- factors affecting the company \'s business , operating results and financial condition . " j.d . edwards develops , markets and supports highly functional enterprise resource planning ( " erp " ) software for managing the supply chain . the company provides the core software products to run an entire business . these software solutions operate on multiple computing platforms and are designed to deliver the solutions that organizations need to maintain control of their businesses 

In [26]:
text_doc[100]

'  the following discussion and analysis of the company \'s operations ,   financial condition and financial performance , as well as material contained   elsewhere herein , includes statements that are not historical facts .   such   statements are forward - looking statements based on the company \'s expectations   and as such , are subject to uncertainty and risk .   for additional   information , the following should be read in conjunction with the selected   financial data and the company \'s consolidated financial statements and notes   to consolidated financial statements which appear elsewhere in this report . the following table sets forth the change in amounts and percentage   change between years of certain revenue , expense and operating items . ( dollars in thousands , except tractor data ) the following table sets forth certain industry operating data of the company . operating revenues operating revenues increased $ dd dd , or dd % , from dd to dd   and $ dd dd , or dd %

In [27]:
my_stop_words = text.ENGLISH_STOP_WORDS.union(["dd"])

In [28]:
tfidf_vect = TfidfVectorizer(stop_words = my_stop_words, min_df=5)
tm= tfidf_vect.fit_transform(text_doc)

In [29]:
type(tm)

scipy.sparse.csr.csr_matrix

In [30]:
tm

<56288x69185 sparse matrix of type '<class 'numpy.float64'>'
	with 48325957 stored elements in Compressed Sparse Row format>

In [32]:
sparse.save_npz('tm_20200212', tm)

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
X_train, X_test, y_train, y_test = train_test_split(tm,labels, test_size=0.2, random_state=42)

In [36]:
labels_train = pd.Series(y_train)
f = 0

aucs = ()
aucs_list = []

precisions = ()

recalls = ()

result = pd.DataFrame()

#exp_weight= []

#kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#embedding_matrix = np.load('embedding_matrix_0721.npy')
#for train_index, test_index in kfold.split(X_train,labels_train):
#     print(' ')

# f = f+1
# print('molde_'+str(f))
# print('train: ',train_index[0], ' / ', train_index[-1])
# print('test: ',test_index[0],' / ', test_index[-1])
#ModelName = './folder(6)/basicM_bestM_l12_'+str(f)
# x_traink, x_val = tm[train_index], tm[test_index]
# y_traink, y_val = labels_train[train_index].values, labels_train[test_index].values

class_weights_dict = {}
class_weights = class_weight.compute_class_weight('balanced',
                                             np.unique(y_train),
                                             y_train)
class_weights_dict[0] = class_weights[0]
class_weights_dict[1] = class_weights[1]
print(class_weights_dict)

svm = LinearSVC(class_weight = class_weights_dict)
svm.fit(X_train, y_train)

#feacture_names = tfidf_vect.get_feature_names()

#plot_coefficients(svm, feacture_names)

decision_values = svm.decision_function(X_test)
auc = roc_auc_score(y_test, decision_values)

pred = svm.predict(X_test)
prfs = precision_recall_fscore_support(y_test, pred)


ap = average_precision_score(y_test, decision_values)
#     if f == 1:
#         pred_1= np.array([test_index,pred,decision_values])
#     else: pred_2= np.array([test_index,pred,decision_values])

dict_temp = {
    'average_precision':ap,
    'auc':auc,'precision_label_0':prfs[0][0],'precision_label_1':prfs[0][1],
    'recall_label_0':prfs[1][0],'recall_label_1':prfs[1][1],
    'fscore_label_0':prfs[2][0],'fscore_label_1':prfs[2][1],
    'support_label_0':prfs[3][0],'support_label_1':prfs[3][1],
}
temp_table = pd.DataFrame(dict_temp,index=[f])
result = pd.concat([temp_table,result])

{0: 0.5045717359150195, 1: 55.18382352941177}


In [37]:
result

,auc,average_precision,fscore_label_0,fscore_label_1,precision_label_0,precision_label_1,recall_label_0,recall_label_1,support_label_0,support_label_1
0,0.907094,0.530316,0.993663,0.467925,0.996754,0.371257,0.990591,0.632653,11160,98
